## Dữ Liệu Chạy Chương Trình

In [14]:
import pandas as pd
import numpy as np
import os
import re
data_base = r"D:\18A1_Nhom_6_Case_11"
data_raw = os.path.join(data_base,"data_raw")
data_clean = os.path.join(data_base,"data_clean")

os.makedirs(data_clean, exist_ok=True)

df_customer = pd.read_csv(os.path.join(data_raw,'customer_profile.csv'))
df_consumption = pd.read_csv(os.path.join(data_raw,'consumption_log.csv'))
df_tariff = pd.read_csv(os.path.join(data_raw,'price_tariff.csv'))

df_customer_clean = pd.read_csv(os.path.join(data_clean,'customer_profile_clean.csv'))
df_consumption_clean = pd.read_csv(os.path.join(data_clean,'consumption_log_clean.csv'))
df_tariff_clean = pd.read_csv(os.path.join(data_clean,'price_tariff_clean.csv'))


#### Nv2

## Thống kê số hộ khách hàng theo house_type.

In [15]:
# Tái sử dụng các DataFrame đã làm sạch từ Nhiệm vụ 1
df_customer_clean = df_customer_clean
df_consumption_clean = df_consumption_clean

# Hợp nhất dữ liệu Tiêu thụ và Hồ sơ Khách hàng (Left Join)
df_merged = pd.merge(df_consumption_clean, df_customer_clean, on='customer_id', how='left')

print("Đã hoàn thành hợp nhất dữ liệu cho Nhiệm vụ 2.")
print(f"Số lượng bản ghi sau khi hợp nhất: {len(df_merged)}")
print("Các trường dữ liệu khả dụng: ['customer_id', 'reading_date', 'consumption_kwh', 'house_type', 'region', ...]")


#Thống kê Số hộ Khách hàng theo Loại nhà
print("\n--- 2.2. Số lượng Khách hàng theo Loại nhà ---")
house_type_count = df_customer_clean['house_type'].value_counts().reset_index()
house_type_count.columns = ['Loại Nhà', 'Số Lượng Khách Hàng']
total_customers = house_type_count['Số Lượng Khách Hàng'].sum()
house_type_count['Tỷ Lệ (%)'] = (house_type_count['Số Lượng Khách Hàng'] / total_customers * 100).round(1)
print(house_type_count)
#Tính Tổng và Trung bình

Đã hoàn thành hợp nhất dữ liệu cho Nhiệm vụ 2.
Số lượng bản ghi sau khi hợp nhất: 66
Các trường dữ liệu khả dụng: ['customer_id', 'reading_date', 'consumption_kwh', 'house_type', 'region', ...]

--- 2.2. Số lượng Khách hàng theo Loại nhà ---
   Loại Nhà  Số Lượng Khách Hàng  Tỷ Lệ (%)
0   Tập Thể                   15       37.5
1  Biệt Thự                   13       32.5
2  Chung Cư                    9       22.5
3   Nhà Phố                    3        7.5


##  Tính tổng và trung bình consumption_kwh theo từng khách hàng

In [16]:
print("\n--- 2.3. Tổng và Trung bình Điện năng tiêu thụ (kWh) theo Khách hàng (Top 5) ---")
customer_consumption_summary = df_consumption_clean.groupby('customer_id')['consumption_kwh'].agg(
    Tổng_Tiêu_Thụ=('sum'),
    Trung_Bình_Tiêu_Thụ=('mean')
).reset_index()
customer_consumption_summary['Trung_Bình_Tiêu_Thụ'] = customer_consumption_summary['Trung_Bình_Tiêu_Thụ'].round(2)
print(customer_consumption_summary.sort_values(by='Tổng_Tiêu_Thụ', ascending=False).head(5))


--- 2.3. Tổng và Trung bình Điện năng tiêu thụ (kWh) theo Khách hàng (Top 5) ---
   customer_id  Tổng_Tiêu_Thụ  Trung_Bình_Tiêu_Thụ
5        KH107          720.0               240.00
12       KH115          590.0               147.50
28       KH133          550.0               183.33
2        KH103          545.0               136.25
21       KH126          450.0               225.00


## Tìm các bản ghi consumption_kwh bất thường (âm, bằng 0 hoặc quá lớn)

In [17]:
#Phân tích Bản ghi Tiêu thụ Bất thường
# Giá trị bằng 0
print("\n--- 2.4. Phân tích Bản ghi Tiêu thụ Bất thường ---")
zero_consumption = df_consumption_clean[df_consumption_clean['consumption_kwh'] == 0]
if not zero_consumption.empty:
    print(f"Số bản ghi có tiêu thụ bằng 0 kWh: {len(zero_consumption)}")
    print("Các bản ghi này cần được kiểm tra để xác định liệu đây có phải là lỗi ghi số hay do khách hàng ngưng sử dụng trong kỳ.")
else:
    print("Không tìm thấy bản ghi có consumption_kwh = 0 (sau khi làm sạch).")
#Giá trị Quá lớn
Q1 = df_consumption_clean['consumption_kwh'].quantile(0.25)
Q3 = df_consumption_clean['consumption_kwh'].quantile(0.75)
IQR = Q3 - Q1
upper_bound = Q3 + 1.5 * IQR
outlier_consumption = df_consumption_clean[df_consumption_clean['consumption_kwh'] > upper_bound]
print(f"\nNgưỡng xác định Outlier (Q3 + 1.5*IQR): {upper_bound:.2f} kWh")
print(f"Số bản ghi được coi là Tiêu thụ BẤT THƯỜNG (quá lớn): {len(outlier_consumption)}")
print("[5 bản ghi tiêu thụ lớn nhất được xác định là Outlier]:")
print(outlier_consumption[['customer_id', 'reading_date', 'consumption_kwh']].sort_values(by='consumption_kwh', ascending=False).head())


--- 2.4. Phân tích Bản ghi Tiêu thụ Bất thường ---
Không tìm thấy bản ghi có consumption_kwh = 0 (sau khi làm sạch).

Ngưỡng xác định Outlier (Q3 + 1.5*IQR): 262.50 kWh
Số bản ghi được coi là Tiêu thụ BẤT THƯỜNG (quá lớn): 12
[5 bản ghi tiêu thụ lớn nhất được xác định là Outlier]:
   customer_id reading_date  consumption_kwh
9        KH107   2024-02-19            300.0
15       KH118   2024-02-17            300.0
20       KH126   2024-02-03            300.0
27       KH130   2024-02-29            300.0
28       KH102   2024-03-12            300.0


## #Thống kê điện năng tiêu thụ trung bình theo region

In [18]:
#Thống kê điện năng tiêu thụ trung bình theo region
print("\n--- 2.5. Điện năng Tiêu thụ Trung bình (kWh) theo Khu vực ---")
region_avg_consumption = df_merged.groupby('region')['consumption_kwh'].mean().round(2).reset_index()
region_avg_consumption.columns = ['Khu Vực', 'TB Tiêu Thụ (kWh)']
print(region_avg_consumption.sort_values(by='TB Tiêu Thụ (kWh)', ascending=False))


--- 2.5. Điện năng Tiêu thụ Trung bình (kWh) theo Khu vực ---
           Khu Vực  TB Tiêu Thụ (kWh)
0         Bắc Ninh             155.91
1           Hà Nội             144.09
3          Đà Nẵng             119.50
2  Tp. Hồ Chí Minh             108.46


## Tính số lần ghi chỉ số (số kỳ) cho mỗi hộ gia đình. 

In [19]:
#Tính số lần ghi chỉ số (số kỳ) cho mỗi hộ gia đình
print("\n--- 2.6. Số lần Ghi chỉ số (Số kỳ) theo Hộ gia đình (Top 5) ---")
reading_count = df_consumption_clean.groupby('customer_id')['reading_date'].count().reset_index()
reading_count.columns = ['Mã Khách Hàng', 'Số Lần Ghi Chỉ Số (Kỳ)']
print(reading_count.sort_values(by='Số Lần Ghi Chỉ Số (Kỳ)', ascending=False).head(5))


--- 2.6. Số lần Ghi chỉ số (Số kỳ) theo Hộ gia đình (Top 5) ---
   Mã Khách Hàng  Số Lần Ghi Chỉ Số (Kỳ)
12         KH115                       4
2          KH103                       4
27         KH132                       4
23         KH128                       4
32         KH138                       3
